In [65]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from keras.layers import *
from keras.models import Sequential
from keras import optimizers
from keras.models import Sequential
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import cv2


## Model For Determining if Fingerprint is real or moulded....?

In [66]:
model = Sequential()
model.add(Conv2D(20,(3,3), padding='same',activation='relu',input_shape=(96,96,1)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(40,(3,3),padding='same',activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(60,(3,3),padding='same',activation='relu'))
model.add(MaxPool2D((2,2))) 
model.add(Conv2D(80,(3,3),padding='same',activation='relu'))

model.add(Flatten())
model.add(Dense(1000,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 96, 96, 20)        200       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 48, 48, 20)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 48, 48, 40)        7240      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 40)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 24, 24, 60)        21660     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 60)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 12, 12, 80)       

In [67]:
model.load_weights("model.h5")

In [68]:
import tensorflow as tf

In [69]:
sgd = tf.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [70]:
img_size = 96

In [71]:
def load_image(img_path):

    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_resize = cv2.resize(img_array, (img_size, img_size))
   
    train_data = np.array(img_resize).reshape(-1, img_size, img_size, 1)
    train_data = train_data / 255.0
                               

    return train_data


In [72]:
model_gender = Sequential()

model_gender.add(Conv2D(filters = 20, kernel_size = (3,3),padding = 'Same',activation ='relu',input_shape = (96,96,1)))
model_gender.add(Conv2D(filters = 40, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model_gender.add(Conv2D(filters = 60, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model_gender.add(Conv2D(filters = 80, kernel_size = (3,3),padding = 'Same',activation ='relu'))

model_gender.add(MaxPool2D(pool_size=(2,2)))

# model_gender.add(Dropout(0.25))
# model_gender.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
# model_gender.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
# model_gender.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model_gender.add(Dropout(0.25))


model_gender.add(Flatten())
model_gender.add(Dense(100, activation = "relu"))
model_gender.add(Dense(10, activation = "relu"))
# model_gender.add(Dense(2, activation = "tanh"))
model_gender.add(Dense(2,activation='softmax'))
# model_gender.add(Dense(1, activation = "sigmoid"))

model_gender.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 96, 96, 20)        200       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 96, 96, 40)        7240      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 96, 96, 60)        21660     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 96, 96, 80)        43280     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 48, 48, 80)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_24 (Dense)             (None, 100)              

In [73]:
model_gender.load_weights("model_gender (1).h5")

In [74]:
model_gender.compile(optimizer = "adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [92]:
def prediction_gender(img_path):
    new_image = load_image(img_path)
    pred = model.predict(new_image)
    labels = np.array(pred)
    if(labels[0][0]>labels[0][1]):
        print("Male")
        s = "Fingerprint may be of Male 👨"
    else:
        print("Female")
        s = "Fingerprint may be of Female 👧"
    return s

In [91]:
def prediction(img_path):
    new_image = load_image(img_path)
    
    pred = model.predict(new_image)
    labels = np.array(pred)
    if(labels[0][0]>labels[0][1]):
        print("Altered Image")
        s = "ALERT !!! THIS IS AN ALTERED IMAGE 😵" 
    else:
        print("Real Image")
        s = "NO Worries !!! It's a Real Image 🙂"
    # p=model.predict_clasess(pred)
    # print(p)
    # print(labels)
    # print(labels[0][0])
    # print(pred)
    return s

In [93]:
from flask import Flask, render_template, request

In [106]:
app = Flask(__name__)

In [107]:
app = Flask(__name__)
@app.route("/", methods=['GET', 'POST'])
def home():
    return render_template('index.html')

@app.route("/predict", methods = ['GET','POST'])

def predict():
    if request.method == 'POST':
        file = request.files['file_1']
        filename = file.filename
        file_path = os.path.join('static/', filename)                       
        file.save(file_path)
        print(filename)
        product = prediction(file_path)
        print(product)
        
    return render_template('predict.html', product = product, user_image = file_path)            

@app.route("/gender", methods = ['GET','POST'])

def gender():
    if request.method == 'POST':
        file = request.files['file_2']
        filename = file.filename
        file_path = os.path.join('static/', filename)                      
        file.save(file_path)
        print(filename)
        product = prediction_gender(file_path)
        print(product)
        
    return render_template('predict.html', product = product, user_image = file_path) 

In [108]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2021 19:57:11] "GET / HTTP/1.1" 200 -


6__M_Left_middle_finger (1).BMP


127.0.0.1 - - [01/Dec/2021 19:58:02] "POST /predict HTTP/1.1" 200 -


Real Image
NO Worries !!! It's a Real Image 🙂
a1.jpeg


...

Altered Image
ALERT !!! THIS IS AN ALTERED IMAGE 😵
r1.jpeg


127.0.0.1 - - [01/Dec/2021 19:59:51] "POST /gender HTTP/1.1" 200 -


Female
Fingerprint may be of Female 👧
a1.jpeg


127.0.0.1 - - [01/Dec/2021 20:00:23] "POST /gender HTTP/1.1" 200 -


Male
Fingerprint may be of Male 👨


In [ ]:
# prediction('a1.jpeg')
# #altered

In [ ]:
# import numpy as np

In [ ]:
# prediction('r1.jpeg')

In [ ]:
# prediction('2.jpg')

In [ ]:
# prediction(r'C:\Users\Shantanu Goyal\Desktop\Flask\Defect-Detection-of-PCB-main\102__M_Right_little_finger_Zcut.BMP')
# altered

In [ ]:
# prediction(r'C:\Users\Shantanu Goyal\Desktop\Flask\103__F_Left_middle_finger_CR.BMP')

In [ ]:
# print(prediction(r'C:\Users\Shantanu Goyal\Desktop\Flask\100__M_Right_ring_finger_CR.BMP'))                                #significance of r"

In [ ]:
# product = prediction(r'C:\Users\nEW u\Flask\Dataset\test\test5\bad (25)k.jpeg')                                                                        #it will accept image directly if both .ipynb and image lie on same place

In [ ]:
# product